#### Pipeline for identifying same cell clusters
This notebook shows some of the work for identifying and labeling same cell clusters. It's not a complete representation of the pipeline, apologies. But I'll do my best to describe the protocol here and the key steps.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

import numpy as np
from matplotlib import pyplot as plt
from vrAnalysis.database import get_database
from vrAnalysis.sessions import B2Session
from vrAnalysis.helpers.debug import Timer
from vrAnalysis.analysis.same_cell_candidates import SameCellProcessor
from vrAnalysis.analysis.same_cell_candidates import (
    plot_correlation_vs_distance,
    plot_plane_pair_histograms,
    plot_cluster_size_distribution,
    plot_distance_distribution,
    plot_roi_removal_analysis,
    ClusterExplorer,
)
from vrAnalysis.analysis.same_cell_candidates.base import SameCellParams, SameCellClusterParameters

sessiondb = get_database('vrSessions')

In [2]:
spks_type = "corrected"

random_choice = True
if random_choice:
    import random
    session = random.choice(sessiondb.iter_sessions(imaging=True, dontTrack=False, session_params={"spks_type": "corrected"}))

else:
    mouse_name = "ATL012"
    date = "2023-01-20"
    session_id = "702"
    session = B2Session.create(mouse_name, date, session_id, dict(spks_type=spks_type))

print(session)

B2Session(mouse_name='CR_Hippocannula7', date='2022-08-26', session_id='702')


In [15]:
cluster_params = SameCellClusterParameters()
processor_params = SameCellParams(
    spks_type=cluster_params.spks_type,
    keep_planes=cluster_params.keep_planes,
    good_labels=cluster_params.good_labels,
    npix_cutoff=cluster_params.npix_cutoff,
)
scp = SameCellProcessor(session, processor_params).load_data()

In [16]:
explorer = ClusterExplorer(scp)

In [ ]:
explorer = explorer.deploy()

In [48]:
from vrAnalysis.helpers.signals import autocorrelation

# Explore how autocorrelation relates to label
activity = session.spks
labels = session.roicat_classifier["class_predictions"]

# Group activity by labels (on axis=1 of activity)
grouped_activity = []
class_name = [session.roicat_classifier["label_to_description"][session.roicat_classifier["id_to_label"][id]] for id in np.unique(labels)]
for label in np.unique(labels):
    grouped_activity.append(activity[:, labels == label])

In [49]:
# Compute autocorrelation for each group
autocorrelations = [autocorrelation(group, axis=0) for group in grouped_activity]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
for ilabel, label in enumerate(np.unique(labels)):
    ax.plot(np.mean(autocorrelations[ilabel], axis=1), label=class_name[ilabel])
ax.legend()
ax.set_xlabel("Lag")
ax.set_ylabel("Autocorrelation")
ax.set_xlim(0, 80)
plt.show()

In [ ]:
clusters = explorer.clusters


In [18]:
show_cluster_size = False
if show_cluster_size:
    plot_cluster_size_distribution(scp, verbose=False)

In [ ]:
show_correlation_vs_distance = True
if show_correlation_vs_distance:
    viewer = plot_correlation_vs_distance(scp)
    viewer.deploy(env="notebook")